<a href="https://colab.research.google.com/github/LordChoupa/test/blob/main/DreamBooth_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 1/ déposer tes photos dans un dossier dans ton drive google puis run la cellule qui permet de te connecter à ton drive
- 2/ modifier les paths et les params du train
- 3/ run le train (sauvegarde de ton modèle vers le path que tu as renseigné) - nécessite une A100 sinon voir la doc pour réduire les besoins gpu
https://huggingface.co/docs/diffusers/training/dreambooth  => chercher le paragraphe 12GB GPU
- 4/ charger le modèle enregistré dans le drive
- 5/ modifier le prompt
- 6/ faire l'inférence - ok avec un Tesla T4 12gb

In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [ ]:
# Connection avec ton drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Requirements

In [ ]:
!pip install git+https://github.com/huggingface/diffusers
!pip install -U -r diffusers/examples/dreambooth/requirements.txt
!pip install accelerate
!pip install -r /content/drive/MyDrive/dreambooth/requirements.txt

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-mo5qg1za
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-mo5qg1za
  Resolved https://github.com/huggingface/diffusers to commit 7a24977ce3f7b406034362c15c17b4159abe7dfd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.21.0.dev0-py3-none-any.whl size=1367521 sha256=6ddda94c3d011e799dfa7816b9e2b9826d7050053dac6cf78e94cfc4d329ab6c
  Stored in directory: /tmp/pip-ephem-wheel-cache-rvj596yr/wheels/f7/7d/99/d361489e5762e3464b3811bc629e94cf5bf5ef44dd5c3c4d52
Successfully built diffusers
ERROR: Could not open requirements file: [Errno 2] No such file or directory:

In [ ]:
from diffusers import DiffusionPipeline, UNet2DConditionModel
from transformers import CLIPTextModel
import torch

**TRAIN**

Le train est à faire une seule fois. Une fois que ton modèle est enregistré sur drive, tu pourras commenter ces lignes

In [ ]:
MODEL_NAME="SG161222/Realistic_Vision_V1.4" #tu peux aussi modif le modèle de départ si besoin ici c'est plutôt pour faire des photos réalistes
INSTANCE_DIR="/content/drive/MyDrive/dreambooth/images" #folder de tes images sur drive
CLASS_DIR="/content/drive/MyDrive/dreambooth/classes" #folder de tes classe sur drive
OUTPUT_DIR="/content/drive/MyDrive/dreambooth/model_output" #folder de ton modèle sur drive
CLASS_PROMPT = "a photo of a person" # indique au modèle le type d'objet créer
INSTANCE_PROMPT = "a photo of fxg person" # crée un concept fxg qui correspond au final à toi

!accelerate launch /content/drive/MyDrive/dreambooth/train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --train_text_encoder \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt=$INSTANCE_PROMPT \
  --class_prompt=$CLASS_PROMPT \
  --mixed_precision="fp16" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_checkpointing \
  --learning_rate=2e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --max_train_steps=800

2023-08-21 12:14:42.260893: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2023-08-21 12:14:47.456759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
08/21/2023 12:14:50 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'timestep_spacing', 'dynamic_thresholding_rat

**INFERENCE**

In [ ]:
# Charger ton modèle qui est sur drive en pointant sur ces éléments unet et text_encoder
model_id = "SG161222/Realistic_Vision_V1.4"
unet = UNet2DConditionModel.from_pretrained("/content/drive/MyDrive/dreambooth/model_output/checkpoint-500/unet")
text_encoder = CLIPTextModel.from_pretrained("/content/drive/MyDrive/dreambooth/model_output/checkpoint-500/text_encoder")

In [ ]:
pipeline = DiffusionPipeline.from_pretrained(model_id, unet=unet, text_encoder=text_encoder, dtype=torch.float16)
pipeline.to("cuda")

vae/diffusion_pytorch_model.safetensors not found


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Keyword arguments {'dtype': torch.float16} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.21.0.dev0",
  "_name_or_path": "SG161222/Realistic_Vision_V1.4",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
nb_images = 4
prompt = "portrait of fxg man superman with cybernetic heavy armor and helmet, sci - fi, glowing intricate, full body, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha"
negative_prompt  = "simple background, duplicate, low quality, lowest quality, bad anatomy, bad proportions, extra digits, lowres, username, artist name, error, duplicate, watermark, signature, text, extra digit, fewer digits, worst quality, jpeg artifacts, blurry"

In [ ]:
image = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=50,
    guidance_scale=6,
    num_images_per_prompt=nb_images
)

for i in range(nb_images):
    display(image[0][i])

  0%|          | 0/50 [00:00<?, ?it/s]